In [31]:
import json
import numpy as np
import os
from pprint import pprint

# prompt_type, task_dir = 'question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1553982706'  # Q-only
# prompt_type, task_dir = 'context_question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1553790696'  # TFIDF
# prompt_type, task_dir = 'context_question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1553901953'  # FastText
### Filtered Workers
# prompt_type, task_dir = 'question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554052233'  # Q-only
# prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554006689'  # TFIDF(Q+O)
# prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554130485'  # TFIDF(O)
# prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554069931'  # Oracle
# prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554072277'  # SL
prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554132868'  # SL-Influence

split = 'middle'
# split = None

# Set useful variables
num_options = 4
options = ['A', 'B', 'C', 'D'][:num_options]
debate_mode_to_option = {'Ⅰ': 'A', 'Ⅱ': 'B', 'Ⅲ': 'C', 'Ⅳ': 'D', 'ⅰ': 'A', 'ⅱ': 'B', 'ⅲ': 'C', 'ⅳ': 'D', None: None}

# Read HIT data
print('# HIT Files:', len(os.listdir(task_dir)))
hit_results = []
num_passed_test = 0
for hit_dir in os.listdir(task_dir):
    if hit_dir.startswith('o_'):
        continue
    num_passed_test += 1
    with open(os.path.join(task_dir, hit_dir, 'custom/data.json'), 'r') as file:
        hit_results.append(json.load(file))
        file.close()
if len(os.listdir(task_dir)) != num_passed_test:
    num_total_tested = len(os.listdir(task_dir)) - num_passed_test
    print('# Passed Test:', num_passed_test, '/', num_total_tested, '=', round((100. * num_passed_test) / num_total_tested, 2), '%')

# HIT Files: 326
# Passed Test: 114 / 212 = 53.77 %


In [32]:
num_valid_hits, num_rejected_hits, num_incomplete_hits = 0, 0, 0
metrics = {}
task_ratings = {i: 0 for i in range(11)}
durations = []
durations_by_worker = []
accuracy_by_worker = {}
max_response_freqs = []
worker_ids = []

for hit_result in hit_results:
    if ((len(hit_result['reject_reasons']) > 0) or
        (len(hit_result['block_reasons']) > 0)):
        num_rejected_hits += 1
        print('| reject_reasons:', hit_result['reject_reasons'], '| block_reasons:', hit_result['block_reasons'])
#         continue
    elif hit_result['feedback'] is None:
        num_incomplete_hits += 1
        continue
    
    worker_ids.append(hit_result['worker_id'])
    num_valid_hits += 1
    if (hit_result['task_rating'] is not None) and (hit_result['task_rating'].isdigit()):
        task_ratings[int(hit_result['task_rating'])] += 1
    for qtype, qtype_accuracy in hit_result['accuracy'].items():
        accuracy_by_worker[qtype] = accuracy_by_worker.get(qtype, []) + [qtype_accuracy]
    
    hit_durations = []
    response_option_counts = {option: 0 for option in options}
    responses = []
    for prompt in hit_result['data']:
        qid = prompt['sample']['qid']
        if (split is not None) and (split not in qid):
            continue
        model_stance = debate_mode_to_option[prompt['sample']['debate_mode']]
        answer = prompt['sample']['eval_labels'][0]
        human_correct = (prompt['response'] == answer)
        assert answer in options, 'Answer must be in options.'
        
        # Calculate metrics
        if qid not in metrics:
            metrics[qid] = {
                option: {
                    'num': 0,
                    'num_correct': 0,
                    'num_correct_debate_mode': 0,
                    'num_incorrect_debate_mode': 0,
                    'num_correct_with_correct_debate_mode': 0,
                    'num_correct_with_incorrect_debate_mode': 0,
                    'num_debate_mode_responses': 0,
                }
                for option in ([None] if model_stance is None else options)
            }
            metrics[qid]['answer'] = answer
        prompt_metrics = metrics[qid][model_stance]
        prompt_metrics['num'] += 1
        prompt_metrics['num_correct'] += human_correct
        if model_stance == answer:
            prompt_metrics['num_correct_with_correct_debate_mode'] += human_correct
            prompt_metrics['num_correct_debate_mode'] += 1
        else:
            prompt_metrics['num_correct_with_incorrect_debate_mode'] += human_correct
            prompt_metrics['num_incorrect_debate_mode'] += 1
        prompt_metrics['num_debate_mode_responses'] += (prompt['response'] == model_stance)
        
        hit_durations.append(prompt['duration'] / 1000.)
        response_option_counts[prompt['response']] += 1
        responses.append(prompt['response'])
    duration = np.median(np.array(hit_durations))
    durations_by_worker.append(duration)
    durations += hit_durations
    response_options_array = np.array(list(response_option_counts.values()))
    response_options_array = response_options_array / response_options_array.sum()
    max_response_freq = response_options_array.max()
    max_response_freqs.append(max_response_freq)
    print('| Time:', round(duration, 1),
          '| Acc:', round(100 * hit_result['accuracy'][prompt_type]),
          '| Max Freq:', round(100 * max_response_freq, 1),
          '| Rate:', hit_result['task_rating'],
          '| Feedback:', hit_result['feedback'])

durations = np.array(durations)
durations_by_worker = np.array(durations_by_worker)
max_response_freqs = np.array(max_response_freqs)
durations.sort()
durations_by_worker.sort()
max_response_freqs.sort()
for qtype in accuracy_by_worker:
    accuracy_by_worker[qtype] = np.array(accuracy_by_worker[qtype])
    accuracy_by_worker[qtype].sort()
print('REJECTED:', num_rejected_hits)
print('INCOMPLETE:', num_incomplete_hits)
print('VALID:', num_valid_hits)
print('Median Question Duration:', np.median(durations))
print('Median Worker Duration:', np.median(durations_by_worker))
print('Median Worker Accuracy:', np.median(accuracy_by_worker[prompt_type]))
print('Median Max Response Freq:', np.median(max_response_freqs))
# pprint(hit_results[0]['data'][0])
pprint(hit_results[0]['worker_id'])

| Time: 16.7 | Acc: 53 | Max Freq: 30.0 | Rate: 1 | Feedback: lower the rate required for the bonus. I used to be in Mensa and if I only got 53% I bet very few people got the bonus.
| Time: 13.6 | Acc: 47 | Max Freq: 30.0 | Rate: 10 | Feedback: Not sure anything can be done to improve it, although some of the questions didn't really make sense.
| Time: 33.0 | Acc: 53 | Max Freq: 40.0 | Rate: 8 | Feedback: Nothing, that was fun!
| Time: 21.3 | Acc: 58 | Max Freq: 40.0 | Rate: 10 | Feedback: n/a
| Time: 11.9 | Acc: 37 | Max Freq: 40.0 | Rate: 10 | Feedback: it's fine as it is
| Time: 17.2 | Acc: 58 | Max Freq: 40.0 | Rate: 8 | Feedback: was actually a fun task. and good pay too
| Time: 16.8 | Acc: 53 | Max Freq: 40.0 | Rate: 10 | Feedback: Can't think of anything
| Time: 17.7 | Acc: 53 | Max Freq: 30.0 | Rate: 8 | Feedback: I think it is meant to be difficult, so I would not make a change.
| Time: 21.9 | Acc: 35 | Max Freq: 45.5 | Rate: 10 | Feedback: Explain the sentences in more simple

| Time: 17.5 | Acc: 53 | Max Freq: 50.0 | Rate: 8 | Feedback: Some of the statements are pure guesswork, maybe make the samples a bit more challenging.
| Time: 22.2 | Acc: 53 | Max Freq: 30.0 | Rate: 10 | Feedback: It would be nice to see the correct answers at some point.
| Time: 20.1 | Acc: 63 | Max Freq: 40.0 | Rate: 8 | Feedback: It's fine as it is. It was interesting.
| Time: 10.6 | Acc: 63 | Max Freq: 40.0 | Rate: 3 | Feedback: Some of the questions just didn't make sense.
| Time: 16.1 | Acc: 45 | Max Freq: 36.4 | Rate: 9 | Feedback: there was one question that had no quote at all
| Time: 7.8 | Acc: 74 | Max Freq: 50.0 | Rate: 5 | Feedback: Na
| Time: 11.4 | Acc: 32 | Max Freq: 50.0 | Rate: 5 | Feedback: Let participants know whether or not they got the correct answer right away.
| Time: 18.0 | Acc: 53 | Max Freq: 40.0 | Rate: 3 | Feedback: Better pay
| Time: 24.9 | Acc: 53 | Max Freq: 30.0 | Rate: 8 | Feedback: I am not sure how to improve it.
| Time: 19.7 | Acc: 58 | Max Freq: 

In [33]:
accuracy_by_sample = []
accuracy_by_sample_correct_debate_mode = []
accuracy_by_sample_incorrect_debate_mode = []
convinced_freqs = []
convinced_freqs_with_correct_debate_mode = []
convinced_freqs_with_incorrect_debate_mode = []
num_evals_by_sample = []
for qid, qid_metrics in metrics.items():
    answer = metrics[qid]['answer']
    for model_stance, prompt in qid_metrics.items():
        if not (model_stance in [None] + options):
            continue

        # Q-only stats
        prompt_metrics = metrics[qid][model_stance]
        num_evals_by_sample.append(prompt_metrics['num'])
        accuracy_by_sample.append(prompt_metrics['num_correct'] / prompt_metrics['num'])
        if model_stance is None:
            continue
        
        # Debater stats
        convinced_freq = prompt_metrics['num_debate_mode_responses'] / prompt_metrics['num']
        if model_stance == answer:
            convinced_freqs_with_correct_debate_mode.append(convinced_freq)
            accuracy_by_sample_correct_debate_mode.append(
                prompt_metrics['num_correct_with_correct_debate_mode'] /
                prompt_metrics['num_correct_debate_mode'])
        else:
            convinced_freqs_with_incorrect_debate_mode.append(convinced_freq)
            accuracy_by_sample_incorrect_debate_mode.append(
                prompt_metrics['num_correct_with_incorrect_debate_mode'] /
                prompt_metrics['num_incorrect_debate_mode'])
        convinced_freqs.append(convinced_freq)

worker_ids = set(worker_ids)
        
num_evals_by_sample = np.array(num_evals_by_sample)
print('Evals per sample:', num_evals_by_sample.mean())
print('Fraction insuffient evals:', (num_evals_by_sample < 5).mean())

convinced_freqs = np.array(convinced_freqs)
print('Convinced:', round(100 * convinced_freqs.mean(), 2), '%')
convinced_freqs_with_correct_debate_mode = np.array(convinced_freqs_with_correct_debate_mode)
print('- Correct debater:', round(100 * convinced_freqs_with_correct_debate_mode.mean(), 2), '%')
convinced_freqs_with_incorrect_debate_mode = np.array(convinced_freqs_with_incorrect_debate_mode)
print('- Incorrect debater:', round(100 * convinced_freqs_with_incorrect_debate_mode.mean(), 2), '%')

accuracy_by_sample = np.array(accuracy_by_sample)
print('Accuracy:', round(100 * accuracy_by_sample.mean(), 2), '%')
accuracy_by_sample_correct_debate_mode = np.array(accuracy_by_sample_correct_debate_mode)
print('- Correct debater:', round(100 * accuracy_by_sample_correct_debate_mode.mean(), 2), '%')
accuracy_by_sample_incorrect_debate_mode = np.array(accuracy_by_sample_incorrect_debate_mode)
print('- Incorrect debater:', round(100 * accuracy_by_sample_incorrect_debate_mode.mean(), 2), '%')

num_target_evals = 5
print('Extra Evals:', round(((100. * (num_evals_by_sample - num_target_evals).sum()) / num_evals_by_sample.sum()), 2), '%')
num_evals_by_sample.sort()
print('Evals per sample distribution:', num_evals_by_sample)

Evals per sample: 5.549019607843137
Fraction insuffient evals: 0.049019607843137254
Convinced: 40.65 %
- Correct debater: 84.54 %
- Incorrect debater: 26.02 %
Accuracy: 55.7 %
- Correct debater: 84.54 %
- Incorrect debater: 46.09 %
Extra Evals: 9.89 %
Evals per sample distribution: [4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


In [34]:
def nps(task_ratings):
    num_ratings = sum(list(task_ratings.values()))
    print(num_ratings)
    if num_ratings == 0:
        return None

    nps_sum_ratings = 0
    sum_ratings = 0
    for score, num_raters in task_ratings.items():
        sum_ratings += num_raters * score
        if score >= 9:
            nps_sum_ratings += num_raters
        elif score <= 6:
            nps_sum_ratings -= num_raters
    return round(100 * (nps_sum_ratings / float(num_ratings)), 2), round((sum_ratings / float(num_ratings)), 2)

print('NPS, Mean:', nps(task_ratings))

111
NPS, Mean: (20.72, 7.69)


In [21]:
durations_by_worker

array([ 7.312 ,  9.326 , 10.364 , 10.967 , 11.18  , 11.252 , 11.446 ,
       11.542 , 11.576 , 12.544 , 12.608 , 12.6555, 12.78  , 13.634 ,
       13.708 , 13.727 , 14.3215, 14.509 , 14.537 , 14.538 , 14.65  ,
       14.887 , 14.928 , 14.975 , 15.074 , 15.264 , 15.545 , 15.748 ,
       15.775 , 15.943 , 15.9795, 15.9865, 16.203 , 16.247 , 16.252 ,
       16.331 , 16.424 , 17.016 , 17.283 , 17.348 , 17.623 , 17.699 ,
       17.77  , 17.78  , 18.1195, 18.157 , 18.223 , 18.256 , 18.274 ,
       18.517 , 18.671 , 18.773 , 19.239 , 19.254 , 19.37  , 19.387 ,
       20.1695, 20.221 , 20.561 , 20.63  , 20.75  , 21.057 , 21.226 ,
       21.314 , 21.4   , 21.637 , 21.789 , 22.047 , 22.052 , 22.103 ,
       22.155 , 22.258 , 22.383 , 22.488 , 22.76  , 22.979 , 22.988 ,
       22.99  , 23.05  , 23.219 , 23.242 , 23.29  , 23.5895, 24.1585,
       24.243 , 24.281 , 24.422 , 24.7485, 25.315 , 25.702 , 26.223 ,
       27.209 , 27.354 , 28.52  , 30.0845, 31.611 , 31.795 , 32.304 ,
       35.076 , 36.9